# Hospital Waste Management LCA

In [189]:
import pandas as pd
import olca_ipc as olca
import uuid
import math
from datetime import datetime
import numpy as np
import sys

# from matplotlib import pyplot as plt
# import matplotlib.mlab as mlab
# from matplotlib import rcParams
# import matplotlib.patches as mpatches
# import seaborn as sns

# params = {'mathtext.default': 'regular' }

# client = olca.Client(8080)
# client

## Preliminary assumptions

For this, we choose to ignore the impacts of transporting waste from the hospital to the various disposal facilities. This is because we are focusing on large hospitals that are located in cities with these facilities easily accessible. Because of the locality of these facilities, we ignore the social, econonmic, and environmental impacts of the transportation.

## Waste Stream Composition Scenarios

In [190]:
# types of waste
# Columns (must sum to 1.0)
# 0 - Plastic, single use
# 1 - Plastic, multi use
# 2 - Paper-based products
# 3 - Glass-based products
# 4 - Bio-hazardous waste
# 5 - Organic waste, non-hazardous

# Data from Aarushi will go here

# Baseline Input Scenario 0
waste_compsition_0 = np.array([0.4, 0.2, 0.2, 0.1, 0.02, 0.08])

# Input Scenario 1
waste_compsition_0 = np.array([0.4, 0.2, 0.2, 0.1, 0.02, 0.08])

# Input Scenario 2
waste_compsition_0 = np.array([0.4, 0.2, 0.2, 0.1, 0.02, 0.08])

## Waste Disposal Method Scenarios

In [191]:
# Waste Disposal Methods
# Rows:
# 0 - Proportion to Landfill
# 1 - Proportion to Recycling
# 2 - Proportion to WtE
# 3 - Proportion to Composting
# 4 - Proportion to Incinerator
# Columns (must sum to 1.0):
# 0 - Plastic, single use
# 1 - Plastic, multi use
# 2 - Paper-based products
# 3 - Glass-based products
# 4 - Bio-hazardous waste
# 5 - Organic waste, non-hazardous

# Data must be decided on by the team

# Worst-case method 0 (all landfill, hazardous waste to incinerators)
waste_treatment_0 = np.array([[1.00, 1.00, 1.00, 1.00, 0.00, 1.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 1.00, 0.00]])

# Waste treatment method 1
waste_treatment_1 = np.array([[1.00, 1.00, 1.00, 1.00, 1.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]])
# Waste treatment method 2
waste_treatment_2 = np.array([[1.00, 1.00, 1.00, 1.00, 1.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]])

## Impact Factor Variables

In [192]:
# Emissions Factors
# Rows:
# 0 - Landfill
# 1 - Recycling
# 2 - WtE (incineration)
# 3 - Composting
# 4 - Incinerator
# Columns:
# 0 - Plastic, single use
# 1 - Plastic, multi use
# 2 - Paper-based products
# 3 - Glass-based products
# 4 - Bio-hazardous waste
# 5 - Organic waste, non-hazardous

# Data must be found for 'impact_cost' and 'impact_co2_eq'. 'impact_community' can be decided upon subjectively

undefi = sys.float_info.max

# Cost of process (values in $/short ton waste)
impact_cost       = np.array([[73.45, 73.45, 73.45, 73.45, 73.45, 73.45],
                              [106.0, 106.0, 106.0, 106.0, 106.0, 106.0],
                              [63.69, 63.69, 63.69, 63.69, 63.69, 63.69],
                              [67.53, 67.53, 67.53, 67.53, 67.53, 67.53],
                              [63.69, 63.69, 63.69, 63.69, 63.69, 63.69]])

# CO2 Emissions of process (ton CO2-eq/short ton waste)
impact_co2_eq     = np.array([[  0.02,   0.02,   0.80,   0.02,   0.48,   0.58],
                              [  0.22,   0.22,   0.07,   0.05, undefi, undefi],
                              [  2.34,   2.34,   0.05,   0.01,   0.05,   0.05],
                              [undefi, undefi, undefi, undefi,   0.17,   0.15],
                              [  2.34,   2.34,   0.05,   0.01,   0.05,   0.05]])

# dry digestate with curing []
# wet digestate with curing []

# Community impact of process
impact_community  = np.array([[1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [1.00, 1.00, 1.00, 1.00, 1.00, 1.00]])

# Social cost of carbon (USD/ton CO2-eq)
social_cost_carbon = 190.0

Municipal Solid Waste (MSW) Landfill	$60.34
MSW Recycling Center	$52.33
MSW Waste-to-Energy Facility	$102.13
Composting Facility	$20-$80
Incinerator Facility	$100-$200

Data sources:

Tipping fees:
* Tipping fee (landfill, PA, 2020): https://assets.website-files.com/602b605a94315c491a4fb4b8/610aed49f0ef85f4566dbe3f_Tipping-Fees-Data-Summary%20Jan%202021.pdf
* Tipping fee (recycling, PA): https://www.phila.gov/media/20210614135759/Municipal-Waste-Management-Plan-appendix-C.pdf
* WtE Facility, PA: https://www.covanta.com/where-we-are/our-facilities/plymouth
* Composting (to energy, PA): https://www.phila.gov/media/20210614135413/Municipal-Waste-Management-Plan-202010.pdf
* https://kleinmanenergy.upenn.edu/research/publications/powering-our-future-with-trash/ 

Emissions data:
* Plastic types (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151894/)
* GHG 2022 Emission factors (https://www.epa.gov/climateleadership/ghg-emission-factors-hub)
* 
* 

Social Cost of Carbon:
* https://www.brookings.edu/articles/what-is-the-social-cost-of-carbon/#:~:text=The%20social%20cost%20of%20carbon%20(SCC)%20is%20an%20estimate%20of,a%20ton%20of%20carbon%20emissions.
* https://www.epa.gov/system/files/documents/2022-11/epa_scghg_report_draft_0.pdf

In [193]:
disposal_method_names = ["Landfill", "Recycling", "Waste-to-Energy", "Composting", "Incineration"]
composition_type_names = ["Plastics, SU", "Plastics, MU", "Paper Products", "Glass Products", "Bio-Hazardous", "Organics"]

In [194]:
def assess_impact(composition, methods, factors, name=None):
    final_impact_matrix = (composition * methods) * factors
    verbose = name != None
    if verbose:
        print("-" * 50)
        print("Analyzing Scenario: " + str(name) + '\n------------------------\n')
        print(final_impact_matrix)
        print("\nTotal " + str(name) + ": " + str(sum(sum(final_impact_matrix)).round(3)) + " per ton waste")
        print("\n\tBreakdown per ton:\n\t------------------------------")
        for i in range(len(disposal_method_names)):
            print('\t' + (disposal_method_names[i] + " " + name).ljust(24) + str(sum(final_impact_matrix[i]).round(3)))
        print()
        for i in range(len(composition_type_names)):
            print('\t' + (composition_type_names[i] + " " + name).ljust(24) + str(sum(final_impact_matrix)[i].round(3)))
        print()
        print("-" * 50)
    return final_impact_matrix

In [195]:
cost_assessment = assess_impact(waste_compsition_0, waste_treatment_0, impact_cost, name='Cost')

--------------------------------------------------
Analyzing Scenario: Cost
------------------------

[[29.38   14.69   14.69    7.345   0.      5.876 ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      1.2738  0.    ]]

Total Cost: 73.255 per ton waste

	Breakdown per ton:
	------------------------------
	Landfill Cost           71.981
	Recycling Cost          0.0
	Waste-to-Energy Cost    0.0
	Composting Cost         0.0
	Incineration Cost       1.274

	Plastics, SU Cost       29.38
	Plastics, MU Cost       14.69
	Paper Products Cost     14.69
	Glass Products Cost     7.345
	Bio-Hazardous Cost      1.274
	Organics Cost           5.876

--------------------------------------------------


In [196]:
co2_assessment = assess_impact(waste_compsition_0, waste_treatment_0, impact_co2_eq, name='CO2')

--------------------------------------------------
Analyzing Scenario: CO2
------------------------

[[0.008  0.004  0.16   0.002  0.     0.0464]
 [0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.001  0.    ]]

Total CO2: 0.221 per ton waste

	Breakdown per ton:
	------------------------------
	Landfill CO2            0.22
	Recycling CO2           0.0
	Waste-to-Energy CO2     0.0
	Composting CO2          0.0
	Incineration CO2        0.001

	Plastics, SU CO2        0.008
	Plastics, MU CO2        0.004
	Paper Products CO2      0.16
	Glass Products CO2      0.002
	Bio-Hazardous CO2       0.001
	Organics CO2            0.046

--------------------------------------------------
